# Uncomment first 2 cells only when doing factory reset. Then comment them out once again

In [ ]:
!pip install streamlit -q  #only use when doin factory reset
!pip install streamlit_folium

     |████████████████████████████████| 7.8MB 3.4MB/s 
     |████████████████████████████████| 174kB 38.5MB/s 
     |████████████████████████████████| 4.2MB 23.3MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 112kB 46.4MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 122kB 45.4MB/s 
ERROR: ipykernel 6.0.1 has requirement ipython>=7.23.1, but you'll have ipython 5.5.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 6.0.1 which is incompatible.
     |████████████████████████████████| 102kB 3.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.12.1 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip  
!unzip -qq ngrok-stable-linux-amd64.zip

--2021-07-07 05:36:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.83.2.115, 3.217.70.31, 52.4.65.107, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.83.2.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  30.9MB/s    in 0.4s    

2021-07-07 05:36:35 (30.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

#go to the link generated from this to see the dashboard

https://8b93ff93f59e.ngrok.io


In [ ]:
#only make changes in this tab and nowhere else. List down the changes that you make by writing down a comment in the line that says changed
%%writefile app.py
import streamlit as st
import folium
import plotly.express as px
import pandas as pd
import plotly.express as px
import numpy as np
from streamlit_folium import folium_static
 
def generate_area(x):
    x=x.split(",")
    try :
      return x[-2]
    except IndexError:
      return x[-1]
 
 
def app_layout():
  #car_info=pd.read_csv("/content/Car-information.csv",encoding="utf_32")
  chargin_station_location=pd.read_csv("/content/electric_vehicle_charging_station_list.csv")
  chargin_station_location['area']=chargin_station_location['address'].apply(lambda x:generate_area(x))
  sales=pd.read_csv("/content/sales_EV.csv")
  st.sidebar.title("Evlytical")
  tab=st.sidebar.radio("Choose the tab",("EV Sales","Charging station","Calculator"))
  if tab=='EV Sales':
    st.title("**EV Sales**")
    temp=sales.groupby("Year")['Total Count'].agg('sum').reset_index()
    temp=temp[temp['Total Count']==max(temp['Total Count'])]
    avg=round(np.mean(sales['Total Count']))
    tot=np.sum(sales['Total Count'])
    col1,col2,col3=st.beta_columns(3)
    with col3:
      st.subheader("Maximum EVs where sold in the year {}".format(temp.iloc[0]['Year']))
      st.subheader(temp.iloc[0]['Total Count'])
    with col2:
      st.subheader("Average EVs sold every month")
      st.subheader(avg)
    with col1:
      st.subheader("Total EVs sold until now ")
      st.subheader(tot)
    col1,col2=st.beta_columns(2)
    typ=col1.radio("EV Type",("Solar","Electric"))
    check=col2.multiselect("Selling Year ",[2017,2018,2019,2020])
    temp=sales[(sales['Year'].isin(check))&(sales['Fuel']==typ)]
    fig=px.line(temp,x='Month',y='Total Count',color='Year',title='Selling of EVs over time ')
    col1,col2=st.beta_columns([3,2])
    col1.plotly_chart(fig)
    grouped_temp=temp.groupby('Year')['Total Count'].agg('sum').reset_index()
    fig=px.bar(grouped_temp,x="Year",y="Total Count",title='Cars sold across the whole year')
    col2.plotly_chart(fig)
  elif tab=="Charging station":
    india_map=folium.Map(zoom_start=6,location=[22,77])
    for index,row in chargin_station_location.iterrows():
      folium.Marker(location=[row['latitude'],row['longitude']],
                          popup=folium.Popup(row['area'])
                          ).add_to(india_map)
    st.title("Charging Station Locations in india")
    folium_static(india_map,height=500,width=1300)
    st.header("Further info about charging stations")
    col1,col2=st.beta_columns(2)
    with col1:
      st.subheader("Type of station")
      type_donut_data=chargin_station_location.groupby('type')['no'].agg('count').reset_index()
      type_donut=px.pie(values=type_donut_data['no'],names=type_donut_data['type'])
      st.plotly_chart(type_donut)
    with col2:
      st.subheader("Distribution by Region")
      region_donut_data=chargin_station_location.groupby('region')['no'].agg('count').reset_index()
      region_donut=px.pie(values=region_donut_data['no'],names=region_donut_data['region'])
      st.plotly_chart(region_donut)
  elif tab=='Calculator':
    st.title("Cost and Fuel Saving Calculator")
    distance=st.slider("Weekly Distance",min_value=0,max_value=500)
    price=st.slider("Fuel Price",min_value=10,max_value=120)
    mileage=20
    st.header("Weekly Savings")
    col1,col2,col3=st.beta_columns(3)
    
    with col1:
      st.subheader("Total Cost Saved")
      st.subheader(round((distance/mileage)*price,2))
    with col2:
      st.subheader("Total Fuel Saved")
      st.subheader("{} Litres".format(distance//15))
    with col3:
      st.subheader("CO2 emissions avoided")
      st.subheader("{:.2f} Kilogram".format((distance/15)*0.121))
    st.header("Yearly Savings")
    col1,col2,col3=st.beta_columns(3)
    
    with col1:
      st.subheader("Total Cost Saved")
      st.subheader(round((distance/mileage)*price*53,2))
    with col2:
      st.subheader("Total Fuel Saved")
      st.subheader("{:.2f} Litres".format((distance//15)*53))
    with col3:
      st.subheader("CO2 emissions avoided")
      st.subheader("{:.2f} Kilogram".format((distance/15)*0.121*53))
 
 
if __name__=='__main__':
  app_layout()

Writing app.py


# run this once and make changes in the app.py and rerun from the app to see the changes in the link above.Do not make any changes over here

In [ ]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.243.180.72:8501

2021-07-07 05:38:12.469 NumExpr defaulting to 2 threads.
  Stopping...
